In [ ]:
import numpy as np
from scipy import signal
from scipy.signal import lfilter
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq, rfft, rfftfreq, fftshift
# import pyfftw
import multiprocessing
import scipy.io as sio
import matplotlib.pyplot as plot
import yaml
import os
import datetime
import glob
from scipy.signal import square, find_peaks, ShortTimeFFT
from scipy.signal.windows import gaussian
from scipy.signal import hilbert
# pyfftw.config.NUM_THREADS = multiprocessing.cpu_count()

%matplotlib inline

In [ ]:
# r = np.load("../sounder/exp/out/2022-11-08/received_450.npz")
# r = np.load("../sounder/out/received_2_2_21m.npz")
# r = np.load("../sounder/out/2022-11-16/received_8.npz")
# r = np.load("../2022-11-16/received_12.npz")
measures = sorted(glob.glob("../../field_data/measurements/*"), key=os.path.getmtime)
# r = np.load(f"../measurements/2023-06-06_16_44/received_150.npz", allow_pickle=True)
print(measures[10])
r = np.load(f"{measures[6]}/received_70.21.npz", allow_pickle=True)

rcv = r["rcv"][0]
sample_rate = 56e6
print(r["rx_time"])

In [ ]:
x = r["ref"][:401]

xcorr = signal.correlate(x, rcv, mode="full", method="fft")
xcorr = np.flip(xcorr) 
xcorr = np.abs(xcorr)
peaks_main, _ = find_peaks(xcorr, distance=401, prominence=40)

clipped_corr = xcorr[peaks_main[0]-10:peaks_main[-1]+200]
peaks, _ = find_peaks(clipped_corr, distance=401, prominence=40)


fig, ax = plt.subplots(2, 1, figsize=(20, 10))
ax[0].plot(clipped_corr)
ax[0].plot(peaks, clipped_corr[peaks], "x")
ax[0].set_yscale('log')
ax[0].set_title("Cross-correlation")



ax[1].plot(rcv[peaks_main[0]:peaks_main[1]])
ax[1].plot(x)
ax[1].legend(["Received", "Transmitted"])

In [ ]:
N = rcv.size
psd = np.fft.fftshift(np.abs(np.fft.fft(rcv)))
f = np.linspace(-sample_rate/2.0, sample_rate/2.0, len(psd))
%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 10]
plt.plot(f, psd)
plt.show()

In [ ]:
Ts = 1/sample_rate
t = np.arange(0, Ts*len(rcv), 1) 

In [ ]:
plot.specgram(rcv, Fs=int(sample_rate))
plot.show()

In [ ]:
f, t, Sxx = signal.spectrogram(rcv, sample_rate)
plt.rcParams['figure.figsize'] = [20, 8]
plt.pcolormesh(t, f, Sxx, shading='gouraud')

plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

In [ ]:
def estimate_and_remove_noise_floor(cir, noise_start_idx, noise_end_idx):
    """
    Estimates the noise floor from the Channel Impulse Response (CIR) and subtracts it.
    
    Parameters:
    - cir: The Channel Impulse Response, as a NumPy array.
    - noise_start_idx: The index in CIR from where noise estimation should start.
    
    Returns:
    - The CIR with the estimated noise floor removed.
    """
    # Estimate the noise floor
    noise_floor = np.mean(np.abs(cir[noise_start_idx:noise_end_idx]))
    
    # Subtract the noise floor from the magnitude of the CIR
    cir_mag_no_noise = np.abs(cir) - noise_floor
    
    # Ensure no negative values
    cir_mag_no_noise[cir_mag_no_noise < 0] = 0
    
    # Retain phase information by recombining the adjusted magnitude with the original phase
    cir_no_noise = cir_mag_no_noise * np.exp(1j * np.angle(cir))
    
    return cir_no_noise


In [ ]:
x = r["ref"][:401]
ofdm_wcp = r["ref"][1704:]

xcorr = signal.correlate(x, rcv, mode="full", method="fft")
xcorr = np.flip(xcorr) 
xcorr = np.abs(xcorr)
peaks_rw, _ = find_peaks(xcorr, distance=401, prominence=40)

print(peaks_rw)

In [ ]:
rcvd_tr  = rcv[peaks_rw[0]:peaks_rw[0]+802] 

In [ ]:
def estimate_frequency_offset(rcv_training, fs):
    """
    Estimate frequency offset using autocorrelation method.

    Parameters:
    - signal: The input signal (numpy array).
    - fs: Sampling frequency of the input signal.

    Returns:
    - Estimated frequency offset in Hz.
    """
    # Calculate autocorrelation of the signal
    hlf_size = len(rcv_training) // 2
    autocorr = np.dot(rcv_training[:hlf_size], np.conj(rcv_training[hlf_size:])) 
    phase_diff = np.angle(autocorr)
    cfo_estimate = (phase_diff / (2 * np.pi)) * fs / 401

    return cfo_estimate

In [ ]:
def moose_alg(samples, fs ):
    num_samples = samples.size
    self_ref_size = num_samples//2
    first_half = np.vstack(samples[:self_ref_size])
    second_half = np.vstack(samples[self_ref_size:])
    phase_offset,_,_,_ = np.linalg.lstsq(first_half, second_half, rcond=None)
    # use phase offset to find frequency
    freq_shift = np.angle(phase_offset)/(2*np.pi)/(1/fs*self_ref_size) 
    freq_shift = np.squeeze(np.array(freq_shift))
    return freq_shift

In [ ]:
def calculate_doppler_shift(freq, speed):
    """
    Calculate the Doppler shift given the frequency and the speed of the moving object.
    
    Parameters:
    - freq: The frequency of the signal.
    - speed: The speed of the moving object.
    
    Returns:
    - The Doppler shift.
    """
    speed_of_light = 3e8
    doppler_shift_freq = freq * (speed_of_light + speed) / speed_of_light
    
    return doppler_shift_freq - freq

In [ ]:
f_off_moose = moose_alg(rcvd_tr, sample_rate)
f_off = estimate_frequency_offset(rcvd_tr, sample_rate)

print(f"Moose alg: {f_off_moose} Hz, Correlation Based: {f_off} Hz")
wo_doppler = f_off_moose - calculate_doppler_shift(3.6e9, -10) 
print(f"Wo dopler: {wo_doppler} Hz")

In [ ]:
Ts = 1/sample_rate
t = np.arange(0, Ts*len(rcv) - Ts, Ts) 
rcv_compansated = rcv * np.exp(-1j*2*np.pi*f_off*t) 

In [ ]:
xcorr = signal.correlate(x, rcv_compansated, mode="full", method="fft")
xcorr = np.flip(xcorr) 
# xcorr = np.abs(xcorr)
peaks, _ = find_peaks(xcorr, distance=401, prominence=40)

In [ ]:
peaks_, _ = find_peaks(xcorr[peaks[0]-2:peaks[0]+360], distance=3, prominence=37)

In [ ]:
peaks_[1:]

In [ ]:
delays = peaks_[1:] - peaks_[0] 

In [ ]:
delays

In [ ]:
for dly in delays:
    dist = dly * 1/sample_rate * 3e8
    print(f"{dist} meters")

In [ ]:
# a2 + 2ab + b2 = (a+b)2 / c2 = a2 + b2 / a + b = c - 10.71  

In [ ]:
corrI = signal.correlate(x.real, rcv.real, mode="full", method="fft")
corrI = corrI/len(x)
corrI = np.flip(corrI)

corrQ = signal.correlate(x.imag, rcv.imag, mode="full", method="fft")
corrQ = corrQ/len(x)
corrQ = np.flip(corrQ)

corrI = corrI**2
corrQ = corrQ**2
corrIQ = np.array([x + y for x, y in zip(corrI, corrQ)])
cir = np.sqrt(corrIQ)
zero_index_corr = np.argmax(xcorr)

In [ ]:
fig, (ax_orig, ax_corr, ax_cir, pdp, constl) = plt.subplots(5, 1, figsize=(15, 15))
ax_orig.plot(x)
# ax_orig.plot(rcv[peaks_rw[0]:peaks_rw[1]-1])
ax_orig.plot(rcv_compansated[peaks[0]:peaks[0]+401])
ax_orig.legend(['Transmitted', 'Received', 'Received Freq Comp.'])
ax_orig.set_title('Received signal vs Transmitted')
ax_orig.set_xlabel("Samples")

zero_index_corr = peaks[0]
corr = xcorr[zero_index_corr-2:zero_index_corr+360]
# corr = xcorr
cir_dB = 10 * np.log10(corr + 1e-8)
ax_corr.plot(cir_dB)
ax_corr.plot(peaks_, cir_dB[peaks_], "x")
ax_corr.set_title('CIR -')
ax_corr.set_xlabel("Time(μs)")

zero_index_cir = np.argmax(cir)
m_cir = cir[zero_index_cir-10:zero_index_cir+500]
m_cir = np.flip(m_cir)
ax_cir.plot(np.linspace(0,(1/sample_rate)*len(m_cir)*1e6,len(m_cir)),np.log10(m_cir))
ax_cir.set_title('CIR')
ax_cir.set_xlabel("Time(μs)")

constl.plot(np.real(rcv_compansated[peaks[0]:peaks[0]+401]), np.imag(rcv_compansated[peaks[0]:peaks[0]+401]), ".")
constl.set_title('Constellation')

pdp.plot( np.linspace(0,1,len(corr)), 20*np.log10(np.abs(corr)**2)+27)
pdp.set_title('PDP')
pdp.set_xlabel("Time(μs)")

ax_orig.margins(0, 0.1)
ax_corr.margins(0, 0.001)
ax_cir.margins(0, 0.001)
pdp.margins(0, 0.001)
constl.margins(0,0.1)

fig.tight_layout()

plt.show()

In [ ]:
import scipy 

K = 512  # Number of subcarriers
P = 512 # Number of pilot subcarriers
CP = K // 4
pilotValue = 1 + 1j
allCarriers = np.arange(K)  # indices of all subcarriers ([0, 1, ... K-1])

pilotCarriers = allCarriers[:: K // P]  # Pilots is every (K/P)th carrier.

pilotCarriers = np.hstack([pilotCarriers, np.array([allCarriers[-1]])])
dataCarriers = np.delete(allCarriers, pilotCarriers)

symbol = np.zeros(K, dtype=complex)  # the overall K subcarriers
symbol[pilotCarriers] = pilotValue  # allocate the pilot subcarriers
symbol[dataCarriers] = 0 + 0j  # allocate the data subcarriers

ofdm_rcv = rcv[zero_index_corr:zero_index_corr+700] / 5

def channelEstimate(OFDM_demod):
    pilots = OFDM_demod[pilotCarriers]  # extract the pilot values from the RX signal
    Hest_at_pilots = pilots / pilotValue # divide by the transmitted pilot values
    
    Hest_abs = scipy.interpolate.interp1d(pilotCarriers, abs(Hest_at_pilots), kind='linear')(allCarriers)
    Hest_phase = scipy.interpolate.interp1d(pilotCarriers, np.angle(Hest_at_pilots), kind='linear')(allCarriers)
    Hest = Hest_abs * np.exp(1j*Hest_phase)
    
    # plt.plot(allCarriers, abs(H_exact), label='Correct Channel')
    plt.stem(pilotCarriers, abs(Hest_at_pilots), label='Pilot estimates')
    plt.plot(allCarriers, abs(Hest), label='Estimated channel via interpolation')
    plt.grid(True); plt.xlabel('Carrier index'); plt.ylabel('$|H(f)|$'); plt.legend(fontsize=10)
    plt.ylim(0,2)
    
    return Hest


def removeCP(signal):
    return signal[CP:]

OFDM_RX_noCP = removeCP(ofdm_rcv)

def DFT(OFDM_RX):
    return np.fft.fft(OFDM_RX)

OFDM_demod = DFT(OFDM_RX_noCP)

Hest = channelEstimate(OFDM_demod)

In [ ]:
plot.specgram(ofdm_rcv, Fs=int(sample_rate))
plot.show()

In [ ]:
pilots = OFDM_demod[pilotCarriers] / 2  # extract the pilot values from the RX signal
Hest_at_pilots = pilots / pilotValue # divide by the transmitted pilot values

Hest_abs = scipy.interpolate.interp1d(pilotCarriers, abs(Hest_at_pilots), kind='linear')(allCarriers)
Hest_phase = scipy.interpolate.interp1d(pilotCarriers, np.angle(Hest_at_pilots), kind='linear')(allCarriers)
Hest = Hest_abs * np.exp(1j*Hest_phase)

# plt.plot(allCarriers, abs(H_exact), label='Correct Channel')
plt.stem(pilotCarriers, abs(Hest_at_pilots), label='Pilot estimates')
plt.plot(allCarriers, abs(Hest), label='Estimated channel via interpolation')
plt.grid(True); plt.xlabel('Carrier index'); plt.ylabel('$|H(f)|$'); plt.legend(fontsize=10)
plt.ylim(0,2)
    

In [ ]:
ht = np.fft.ifft(Hest_at_pilots, K)
plt.plot(ht)
plt.show()

In [ ]:
import numpy as np

def zadoff_chu_sequence(N, root):
    """
    Generate a Zadoff-Chu sequence of length N with a given root index.
    """
    n = np.arange(N)
    seq = np.exp(-1j * np.pi * root * n * (n + 1) / N)
    return seq

def freq_offset_estimation(received_signal, N, root):
    """
    Estimate the frequency offset using a Zadoff-Chu sequence.
    """
    # Generate the Zadoff-Chu sequence
    sequence = zadoff_chu_sequence(N, root)
    
    # Compute the correlation between the received signal and the sequence
    corr = np.conj(received_signal) * sequence
    
    # Compute the angle of the correlation values
    angles = np.angle(corr)
    
    # Compute the frequency offset estimate
    freq_offset_estimate = np.mean(np.diff(angles)) / (2 * np.pi)
    
    return freq_offset_estimate

# Example usage
N = 64  # Length of the sequence
root = 25  # Root index for the Zadoff-Chu sequence

# Generate a random received signal with frequency offset
freq_offset = 0.1  # Normalized frequency offset
received_signal = np.exp(1j * 2 * np.pi * freq_offset * np.arange(N))

# Estimate the frequency offset
freq_offset_estimate = freq_offset_estimation(received_signal, N, root)

print(f"True frequency offset: {freq_offset}")
print(f"Estimated frequency offset: {freq_offset_estimate}")

In [ ]:
# imports go here
import numpy as np
import matplotlib.pyplot as plt
 
# functions go here
def moose_alg(samples, fs ):
    num_samples = samples.size
    self_ref_size = num_samples//2
    first_half = np.matrix(samples[:self_ref_size])
    second_half = np.matrix(samples[self_ref_size:])
    phase_offset,_,_,_ = np.linalg.lstsq(first_half.transpose(), second_half.transpose() )
    # use phase offset to find frequency
    freq_shift = np.angle(phase_offset)/2/np.pi/(1/fs*self_ref_size)
    return freq_shift
 
# main thing goes here
if __name__ == '__main__':
    # some params
    freq_offset = 5
    fs = 10000.0
    # establish our barker codes
    bc13 = np.array([1, 1, 1, 1, 1, -1, -1, 1, 1, -1, 1, -1, 1])
    # let's make our training sequence bc13, flip bc13, bc13, flip bc13
    training = np.hstack( (bc13, bc13) )
    training = np.hstack( (training, training))
    # mess up our training using frequency offset and noise
    freq_offset = 5
    n = np.arange( training.size )
    time = n/fs
    freq_off_vec = np.exp(1j*2*np.pi*freq_offset*time)
    received = training*freq_off_vec + np.random.randn(training.size)*.1
    # pass what we have to moose algorithm
    freq_shift = moose_alg( received, fs )
    print(freq_shift)